In [1]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime, date
import time


In [79]:
    thisGw = 33

    def getGwStart():
        gw = thisGw
        liste = [5, 9, 13, 17, 21, 25, 29, 33]
        for obj in liste:
            if gw < obj:
                return obj - 4 
        else:
            return 33     

    gws = getGwStart()

    def gwEnd ():
        gw = gws
        if gw == 33:
            return 38
        else:
            return gw + 3
    gwE = gwEnd()

In [90]:
def getBootstrapTeams():
    url4 = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    r4 = requests.get(url4)
    json = r4.json()
    gameweek_df = pd.DataFrame(json['elements'])
    teams = gameweek_df[['id', 'team', 'element_type']]
    teams.set_index('id', inplace = True)
    return teams

teams = getBootstrapTeams()
def getTeamList():
        url2 = 'https://fantasy.premierleague.com/api/leagues-classic/627607/standings/'
        r2 = requests.get(url2)
        json2 = r2.json()
        standings_df = pd.DataFrame(json2['standings'])
        league_df = pd.DataFrame(standings_df['results'].values.tolist())
        return league_df [['entry', 'player_name']]

teamsList = getTeamList()

In [113]:
url = 'https://fantasy.premierleague.com/api/entry/3378570/history/'
teamPoints = requests.get(url).json()['current']
teamPoints[4 - 1]['total_points']


250

In [184]:
 def getRoundPoints(slutter):
        starter = slutter - 5
        slutter = starter + 4

        result = []
        navn = []
        # Finner scoren til alle spillerne i parmeterintervallet
        
        for i in range (len(teamsList)):
            url = 'https://fantasy.premierleague.com/api/entry/' + str(teamsList.iat[i,0]) + '/history/'
            r = requests.get(url)
            json = r.json()
            teamPoints_df = pd.DataFrame(json['current'])
            result.append(teamPoints_df['points'][starter:slutter].sum() - 
                        teamPoints_df['event_transfers_cost'][starter:slutter].sum())
            navn.append(teamsList.iat[i,1])
        
        samlet = pd.DataFrame(result, navn)
        samlet.reset_index(inplace = True)
        maxClm = samlet.loc[samlet[0].argmax()]
        
        return maxClm



def getWinners():
    nyRunde = [5, 9, 13, 17, 21, 25, 29, 33, 37]
    rundevinnere = []
    gwIntervall = ["1 → 4", "5 → 8", "9 → 12", "13 → 16", "17 → 20",
    "21 → 24", "25 → 28", "29 → 32", "33 → 36", "37 → 38"]
    for obj in nyRunde:
        if gws < obj:
            break
        if gws >= obj:
            rundevinnere.append(getRoundPoints(obj))
            
    result = pd.DataFrame(rundevinnere)
    result.insert(0, 'Runde', gwIntervall[:(len(result))], True)
    result.columns = ['GW', 'Vinner', 'Poeng']
    return result

def getWinners2():
    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    events = requests.get(url).json()['events']
    nyRunde = [(1,4), (5,8), (9,12), (13,16), (17,20), (21,24), (25,28), (29,32), (33,38)]
    rundevinnere = []
    result = []
    i = 0
    gwIntervall = ["1 → 4", "5 → 8", "9 → 12", "13 → 16", "17 → 20",
    "21 → 24", "25 → 28", "29 → 32", "33 → 38"]
    for rndS, rndE in nyRunde:
        if events[rndE - 1]['data_checked']:
            rundevinnere.append(getRoundWinners(rndS, rndE))
    for points, win in rundevinnere:
        result.append({
            'GW': gwIntervall[i],
            'Vinner': win,
            'Poeng': points
        })
        i += 1
    return result

getWinners2()

[{'GW': '1 → 4', 'Vinner': 'Tor Aimar Carlsen', 'Poeng': 257},
 {'GW': '5 → 8', 'Vinner': 'Espen Sævig', 'Poeng': 259},
 {'GW': '9 → 12', 'Vinner': 'Espen Sævig', 'Poeng': 271},
 {'GW': '13 → 16', 'Vinner': 'Vegard Vestvik', 'Poeng': 261},
 {'GW': '17 → 20', 'Vinner': 'Vegard Vestvik', 'Poeng': 292},
 {'GW': '21 → 24', 'Vinner': 'Oda Haugane', 'Poeng': 307},
 {'GW': '25 → 28', 'Vinner': 'Vegard Vestvik', 'Poeng': 324}]

In [185]:
getWinners2()

[{'GW': '1 → 4', 'Vinner': 'Tor Aimar Carlsen', 'Poeng': 257},
 {'GW': '5 → 8', 'Vinner': 'Espen Sævig', 'Poeng': 259},
 {'GW': '9 → 12', 'Vinner': 'Espen Sævig', 'Poeng': 271},
 {'GW': '13 → 16', 'Vinner': 'Vegard Vestvik', 'Poeng': 261},
 {'GW': '17 → 20', 'Vinner': 'Vegard Vestvik', 'Poeng': 292},
 {'GW': '21 → 24', 'Vinner': 'Oda Haugane', 'Poeng': 307},
 {'GW': '25 → 28', 'Vinner': 'Vegard Vestvik', 'Poeng': 324}]

In [179]:
def getRoundWinners(roundStart, roundEnd):
    result = []
    for i in range (len(teamsList)):
        url = 'https://fantasy.premierleague.com/api/entry/' + str(teamsList.at[i,'entry']) + '/history/'
        teamPoints = requests.get(url).json()['current']
        if roundEnd == 4:
            result.append((
                teamPoints[roundEnd - 1]['total_points'],
                teamsList.at[i,'player_name']
                ))
        else:
            result.append(
                (teamPoints[roundEnd - 1]['total_points'] 
                - teamPoints[roundStart - 2]['total_points'],
                teamsList.at[i,'player_name'],))
    return max(result)

In [177]:
max(getRoundWinners(5, 8))

(259, 'Espen Sævig')

In [164]:
getRoundWinners(1, 4)

[('Vegard Vestvik', 242),
 ('Oda Haugane', 240),
 ('Tor Aimar Carlsen', 257),
 ('Espen Sævig', 250),
 ('Endre Bratland Skurtveit', 254),
 ('Dan Erik Linde', 207),
 ('Stian Linde', 226),
 ('Eirik Rode', 210),
 ('Simen Madsen', 232),
 ('Hogne Vestvik', 165)]

In [180]:
pd.DataFrame(getRoundWinners(5, 8))

,0
0,259
1,Espen Sævig


In [75]:
url2 = 'https://fantasy.premierleague.com/api/bootstrap-static/'
events2 = requests.get(url2).json()['events']
events2[37]['data_checked']

False

In [39]:
print(getGwStart())
print(gwEnd())

33
38


In [108]:
thisGw = 30
def dateAndTime(dataTime):
    return datetime.strptime(dateTime, "%Y-%m-%dT%H:%M:%SZ")

In [72]:
dateAndTime()

'2021-04-03T11:30:00Z'

In [176]:
#d = datetime.strptime('2021-04-03T11:30:00Z', "%Y-%m-%dT%H:%M:%SZ")
d = datetime.strptime('2021-04-03T11:30:00Z', "%Y-%m-%dT%H:%M:%SZ")
print(d)

2021-04-03 11:30:00


In [179]:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()
d.strftime('%m-%d')
d.strftime('%d-%m-%H:%M')
d.replace(tzinfo=from_zone)
central = d.astimezone(to_zone)
central.strftime('%d-%m-%H:%M')




'03-04-11:30'

In [184]:
def getDateTime(utc_datetime):
    utc_datetime = datetime.strptime(utc_datetime, "%Y-%m-%dT%H:%M:%SZ")
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

In [185]:
datetime_from_utc_to_local('2021-04-03T11:30:00Z')

datetime.datetime(2021, 4, 3, 13, 30)

In [61]:
def isGameInPlay(teamID):

    url = 'https://fantasy.premierleague.com/api/fixtures/?event='+ str(thisGw)
    fixtures = requests.get(url).json()
    for fix in fixtures:
        if teamID == fix['team_a'] or teamID == fix['team_h']:
            return fix['started'] and not fix['finished_provisional']

In [62]:
isGameInPlay(19)

False

In [35]:
test = not False

In [36]:
test

True